# Qiskit Serverless Program Uploader

This notebook uploads a program that generates random quantum circuits and transpiles them using Qiskit Serverless.

In [35]:
%pip install qiskit-serverless -U

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import necessary libraries
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_catalog import QiskitServerless, QiskitFunction
from qiskit_serverless import ServerlessClient

USE_RUNTIME_SERVICE = False

service = None
if USE_RUNTIME_SERVICE:
    service = QiskitRuntimeService(
        channel='ibm_quantum',
        instance='ibm-q/open/main',
        token='02ca823712f3a731ef51db73c66203d45b20ac8a10e0ffac63391a4bc6dbab7e448afe53a45aa49e6cee9296404d043e41aa644a692232c20dba2c535ba4ed64',
        verify=False
    )
print(service) 


None


### Authentication Approach 1- using Serverless Client (reference: https://qiskit.github.io/qiskit-serverless/examples/01_vqe.html)

In [48]:
import os
serverless=ServerlessClient(token=os.environ.get("GATEWAY_TOKEN", "awesome_token"),
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),)
serverless

QiskitServerlessException: Cannot verify token.

### Authentication Approach 2: using IBMServerlessClient (reference:https://github.com/qiskit-community/ibm-quantum-challenge-2024/blob/main/solutions/lab_3/lab-3-serverless-solution.ipynb)

In [38]:
IBMServerlessClient.save_account("02ca823712f3a731ef51db73c66203d45b20ac8a10e0ffac63391a4bc6dbab7e448afe53a45aa49e6cee9296404d043e41aa644a692232c20dba2c535ba4ed64","phalak2684@gmail.com",True)
serverless=IBMServerlessClient("02ca823712f3a731ef51db73c66203d45b20ac8a10e0ffac63391a4bc6dbab7e448afe53a45aa49e6cee9296404d043e41aa644a692232c20dba2c535ba4ed64")

InvalidAccountError: "Invalid `instance` value. Expected a non-empty string, got 'None'. If using the ibm_quantum channel, please specify the channel when saving your account with `channel = 'ibm_quantum'`."

## Implementation

In [ ]:
serverless = QiskitServerless()

# Define the function for uploading
transpile_remote_demo = QiskitFunction(
    title="transpile_remote_serverless",
    entrypoint="transpile_remote.py",
    working_dir="./source_files/"
)

# Upload the program
serverless.upload(transpile_remote_demo)

# Check if it was uploaded successfully
print(serverless.list())

In [ ]:
depths = [5, 10, 15]  
optimization_levels = [0, 1, 2, 3]  

from source_files.generate_random import generate_random_circuits
circuits = generate_random_circuits(depths)

depth_results = {level: [] for level in optimization_levels}
for opt_level in optimization_levels:
    job = serverless.run(
        "transpile_remote_serverless",
        inputs={"circuits": circuits, "backend_name": "ibmq_qasm_simulator", "optimization_level": opt_level}
    )

    result = job.result()
    for circuit_data in result['transpiled_circuits']:
        transpiled_circuit, reduced_depth = circuit_data
        depth_results[opt_level].append(reduced_depth)

In [ ]:
for level in optimization_levels:
    print(f"Optimization Level {level}: Reduced Depths: {depth_results[level]}")

min_depths = [min(depth_results[level]) for level in optimization_levels]
for i in range(1, len(min_depths)):
    if min_depths[i] >= min_depths[i - 1]:
        print(f"Minimum optimization level after which no further optimizations occur: {optimization_levels[i-1]}")
        break
else:
    print("All optimization levels provided further reductions.")

## Final observations:
- This task made me explore QiskitServerless and its importance in more detail.
- I learned more about time complexity which increases (or remains the same) with increase in optimization levels. 
- The goal of this task was to find the minimum optimization level after which the depth of the circuit isn't reduced further.